In [1]:
import process as p
import pandas as pd
import dmc

In [2]:
df = p.processed_data()
df = p.shuffle(df)

In [3]:
start, end, split = pd.Timestamp('2014-1-1'), pd.Timestamp('2014-12-31'), pd.Timestamp('2014-10-1')
df.orderDate = pd.to_datetime(df.orderDate)
mask = (df.orderDate >= start) & (df.orderDate <= end)
df_full = df[mask]
te_size = 5000
tr_size = 5000
X, Y, fts = dmc.transformation.transform_preserving_header(df_full, scaler=dmc.transformation.scale_features, binary_target=True)
train = X[:tr_size], Y[:tr_size]
test = X[tr_size:tr_size+te_size], Y[tr_size:tr_size+te_size]

/home/bpm1/BigData/run-dmc/venv/lib/python3.4/site-packages/pandas/core/generic.py:2701: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
#res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)

In [5]:
'''Calculate impact of features for n iterations'''
impact_map = dict()
feature_set = set(fts)
for feature in fts:
        impact_map[feature] = 0.
for i in range(1):
    print(i)
    res_tree = dmc.evaluation.evaluate_features_leaving_one_out(train[0], train[1], test[0], test[1], fts, dmc.classifiers.DecisionTree)
    for feature in feature_set:
            impact_map[feature] = impact_map[feature] + res_tree.decrement[feature]
import operator
sorted_impact_list = sorted(impact_map.items(), key=operator.itemgetter(1))
print(sorted_impact_list)


0
[('t_isGift', -0.0031999999999999806), ('surplusArticleSizeQuantity', -0.0026000000000000467), ('t_order_totalPrice', -0.0022000000000000908), ('sizeCode', -0.0020000000000000018), ('t_isTypePants', -0.0020000000000000018), ('deviceID', -0.0020000000000000018), ('orderWeekOfYear', -0.0020000000000000018), ('t_order_article_sameArticlesCount_DiffColor', -0.0018000000000000238), ('t_voucher_isValueVoucher', -0.0015999999999999348), ('t_order_boughtArticleCount', -0.0015999999999999348), ('t_order_sameArticlesCount_DiffColor', -0.0015999999999999348), ('orderMonth', -0.0010000000000000009), ('t_singleItemPrice', -0.00080000000000002292), ('orderTotalDay', -0.000200000000000089), ('orderDay', 0.0), ('t_order_duplicateCount', 0.00040000000000006697), ('surplusArticleQuantity', 0.00060000000000004494), ('t_customer_voucherCount', 0.00060000000000004494), ('t_article_availableSizes', 0.0010000000000000009), ('orderWeek', 0.0010000000000000009), ('t_isOneSize_A', 0.0011999999999999789), ('t_

In [6]:
res_tree[(res_tree.decrement < 0)]

,decrement,precision
orderTotalDay,-0.0002,0.6908
sizeCode,-0.0020,0.6926
t_isTypePants,-0.0020,0.6926
t_order_article_sameArticlesCount_DiffColor,-0.0018,0.6924
surplusArticleSizeQuantity,-0.0026,0.6932
t_order_boughtArticleCount,-0.0016,0.6922
t_isGift,-0.0032,0.6938
t_order_sameArticlesCount_DiffColor,-0.0016,0.6922
deviceID,-0.0020,0.6926
orderWeekOfYear,-0.0020,0.6926


In [8]:
kick_features = 5
for i in range(kick_features):
    improvement_without_feature = dmc.evaluation.evaluate_without_one_feature(train[0], train[1], test[0], test[1], fts, sorted_impact_list[i][0], dmc.classifiers.DecisionTree, 10)
    print(improvement_without_feature)

Improvement without t_isGift: -0.00045999999999990493
Improvement without surplusArticleSizeQuantity: 0.00042000000000008697
Improvement without t_order_totalPrice: -0.000580000000000136
Improvement without sizeCode: 0.0017000000000000348
Improvement without t_isTypePants: 0.00033999999999989594
